In [1]:
from hypernodes import DualNode, Pipeline, node

In [2]:
from typing import List


class Encoder:
    def __init__(self, model_name: str):
        self.model_name = model_name

    def encode(self, text: str) -> List[float]:
        return [0.1, 0.2, 0.3]

    def encode_batch(self, texts: List[str]) -> List[List[float]]:
        return [[0.1, 0.2, 0.3] for _ in texts]


In [3]:
@node(output_name="encoded_text")
def process(text: str, encoder: Encoder) -> List[float]:
    return encoder.encode(text)


pipeline = Pipeline(nodes=[process])

In [4]:
pipeline.visualize()

In [5]:
results = pipeline.run(
    inputs={"text": "Hello, world!", "encoder": Encoder(model_name="test")}
)

In [6]:
results

{'encoded_text': [0.1, 0.2, 0.3]}

In [7]:
results = pipeline.map(
    inputs={
        "text": ["Hello, world!", "Hey! World"],
        "encoder": Encoder(model_name="test"),
    },
    map_over="text",
)

# DualNode Example: Batch-Optimized Nodes

DualNode allows you to define two implementations:
- **singular**: For single-item execution (used in `.run()` and type hints)
- **batch**: For optimized batch execution (used in `.map()` with DaftEngine)


In [8]:
try:
    from daft import Series
except ImportError:
    Series = list  # Fallback for type hints


# Define singular and batch functions
def encode_singular(text: str, encoder: Encoder) -> List[float]:
    """Process single item"""
    return encoder.encode(text)


def encode_batch(texts: Series, encoder: Encoder) -> Series:
    """Process batch of items - optimized"""
    return Series.from_pylist([encoder.encode(t) for t in texts.to_pylist()])


# Create DualNode
dual_encode_node = DualNode(
    output_name="encoded_text",
    singular=encode_singular,
    batch=encode_batch,
)


In [9]:
# Create pipeline with DualNode
dual_pipeline = Pipeline(nodes=[dual_encode_node])
dual_pipeline.visualize()


In [10]:
# Test with .run() - uses singular function
encoder = Encoder(model_name="test")
result = dual_pipeline.run(inputs={"text": "Hello, world!", "encoder": encoder})
print("Single item result:", result)


Single item result: {'encoded_text': [0.1, 0.2, 0.3]}


In [11]:
# Test with .map() - uses batch function (optimized!)
results = dual_pipeline.map(
    inputs={
        "text": ["Hello", "World", "Test"],
        "encoder": encoder,
    },
    map_over="text",
)
print("Batch results:", results)


Batch results: [{'encoded_text': [0.1, 0.2, 0.3]}, {'encoded_text': [0.1, 0.2, 0.3]}, {'encoded_text': [0.1, 0.2, 0.3]}]


In [12]:
results

[{'encoded_text': [0.1, 0.2, 0.3]},
 {'encoded_text': [0.1, 0.2, 0.3]},
 {'encoded_text': [0.1, 0.2, 0.3]}]